In [10]:
import pandas as pd
import json
import snowflake.connector
from configs.config import snowflake_conn_prop_local as snowflake_conn_prop
from snowflake.snowpark.session import Session

session = Session.builder.configs(snowflake_conn_prop).create()

source_table = "CITIBIKE_2.VALIDATION.TRIPS_VALIDATION_RAW"
destination_table = "CITIBIKE_2.VALIDATION.TRIPS_VALIDATION_CLEAN"

# Read data from the specified Snowflake table into a DataFrame
df_sql = session.table(source_table)
data = df_sql.collect()
df = pd.DataFrame(data) 

json_object = json.loads(df['RUNVALIDATION'].iloc[6])

# Import the necessary libraries
import pandas as pd

# Initialize a list to store the extracted information
extracted_data = []

# Initialize a dictionary to store information for different expectation types
expectation_data = {}

# Assuming json_object contains the provided JSON data
for item in json_object:
    results = item.get("results", [])  # Get the results list or an empty list if not present
    for result in results:
        expectation_type = result.get("expectation_config", {}).get("expectation_type")
        
        # Extract expectation_config and remove batch_id if it exists
        expectation_config = result.get("expectation_config", {}).get("kwargs", {})
        expectation_config.pop("batch_id", None)  # Remove batch_id

        observed_values = result.get("result", {}).get("observed_value")
        success_status = result.get("success", False)
        runtime = item.get("meta", {}).get("run_id", {}).get("run_time")
            
        # Append the extracted information to the list
        extracted_data.append({
            "EXPECTATION_TYPE": expectation_type,
            "RUNTIME": runtime,
            "EXPECTATION_CONFIG": expectation_config,
            "OBSERVED_VALUE": observed_values,
            "SUCCESS_STATUS": success_status,
            "RUNTIME": runtime
        })
            
        # Store the information in a dictionary using the expectation_type as the key
        expectation_data.setdefault(expectation_type, []).append({
            "EXPECTATION_TYPE": expectation_type,
            "RUNTIME": runtime,
            "EXPECTATION_CONFIG": expectation_config,
            "OBSERVED_VALUE": observed_values,
            "SUCCESS_STATUS": success_status,
        })

df = pd.DataFrame(extracted_data)

from snowflake.snowpark.session import Session
from configs.config import snowflake_conn_prop_local as snowflake_conn_prop
import json

# Create a session object with Snowflake
session = Session.builder.configs(snowflake_conn_prop).create()

# Assuming df is your DataFrame with data
table_name = 'citibike_2.validation.TRIPS_VALIDATION_CLEAN'

# Process each row and create a list of dictionaries representing each row
data_to_insert = []
for index, row in df.iterrows():
    expectation_type = row['EXPECTATION_TYPE']
    runtime = row['RUNTIME']
    expectation_config = json.dumps(row['EXPECTATION_CONFIG'])
    observed_values = json.dumps(row['OBSERVED_VALUE'])
    success_status = row['SUCCESS_STATUS']

    data_to_insert.append({
        "EXPECTATION_TYPE": expectation_type,
        "RUNTIME": runtime,
        "EXPECTATION_CONFIG": expectation_config,
        "OBSERVED_VALUE": observed_values,
        "SUCCESS_STATUS": success_status
    })

# Create a DataFrame from the list of dictionaries
df_to_insert = session.create_dataframe(data_to_insert)

# Write the DataFrame to the Snowflake table
df_to_insert.write.mode("append").save_as_table(table_name)

# Print the number of rows inserted
print(f"Number of rows inserted: {len(data_to_insert)}")

# Close the session
session.close()


Number of rows inserted: 3
